# 🔬 SmartLock Admin API Testing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/spk-alex/smartlock-api-colab/blob/main/smartlock_api_testing_colab.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-View%20Source-blue?logo=github)](https://github.com/spk-alex/smartlock-api-colab)

**🚀 Test the SmartLock Admin API interactively - no installation required!**

## What This Notebook Does

This notebook demonstrates the complete SmartLock API workflow:
1. ✅ **Create a user** - Register a new user in the system
2. 🔑 **Generate JWT token** - Create authentication for API calls
3. 🏠 **Create a smart lock** - Add a new smart lock for the user
4. 👥 **Add a resident** - Add a resident to the smart lock
5. 📊 **Test all endpoints** - Verify everything works correctly
6. 🧹 **Automatic cleanup** - Remove test data safely

## 🛠️ How to Use

1. **Choose environment** in the next cell (Development or Staging)
2. **Click "Runtime" → "Run all"** to execute all tests
3. **Watch the progress** as each test runs
4. **Review results** at the end

## 🔒 Safety

- ✅ Uses **test environments only** (never production)
- ✅ Creates **temporary test data** (unique per run)
- ✅ **Automatic cleanup** removes test data
- ✅ **No real user data** involved

*Last updated: 2025-06-10 13:42:08 UTC from CI/CD pipeline*

---


In [ ]:
#@title 🎯 Choose API Environment { display-mode: "form" }
#@markdown Select which environment you want to test:

ENVIRONMENT = "staging" #@param ["development", "staging"]
CUSTOM_BASE_URL = "" #@param {type:"string"}
API_TIMEOUT = 45 #@param {type:"slider", min:10, max:120, step:5}
SHOW_DETAILED_LOGS = True #@param {type:"boolean"}

# Environment URLs
ENVIRONMENT_URLS = {
    "development": "https://restate-dev.api.spkey.co",
    "staging": "https://restate-staging.api.spkey.co"
}

# Use custom URL if provided, otherwise use environment default
if CUSTOM_BASE_URL.strip():
    BASE_URL = CUSTOM_BASE_URL.strip()
    print(f"🌐 Using custom URL: {BASE_URL}")
else:
    BASE_URL = ENVIRONMENT_URLS[ENVIRONMENT]
    print(f"🌐 Using {ENVIRONMENT} environment: {BASE_URL}")

print(f"⏱️ API timeout: {API_TIMEOUT} seconds")
print(f"📋 Detailed logs: {'Enabled' if SHOW_DETAILED_LOGS else 'Disabled'}")
print(f"\n✅ Configuration ready! You can now run the tests.")

# Set CI/CD compatibility variables
CI_COMMIT_SHA = "colab-user"
CI_PIPELINE_ID = "colab-session"


In [ ]:
# Install required packages (only in Colab)
print("📦 Installing required packages...")
!pip install requests pandas -q
print("✅ Packages installed!")

# Import libraries
import requests
import json
import base64
import uuid
import random
import string
import time
from datetime import datetime, timedelta
from typing import Dict, Any, Optional
import pandas as pd
from pprint import pprint

print("📚 All libraries imported successfully!")


# SmartLock Admin API Testing Notebook

This notebook tests the SmartLock Admin API endpoints using the public Cloudflare domain.
It demonstrates the complete flow:
1. Create a user
2. Generate JWT token for authentication
3. Create a smart lock for the user
4. Add a resident to the lock
5. Test getting locks for the user
6. Test getting residents for the lock

## Configuration and Imports

## Parameters (for CI/CD execution)

These parameters can be overridden by papermill during CI/CD execution:

## Configuration Parameters

Modify these parameters for different test scenarios:

## JWT Token Generation

Create a simple JWT token for testing authentication:

In [ ]:
def create_test_jwt_token(user_id: str, expires_in_hours: int = 24) -> str:
    """
    Create a simple JWT token for testing purposes.
    
    Note: This creates an unsigned JWT token for testing.
    In production, tokens should be properly signed and validated.
    """
    # JWT Header
    header = {
        "alg": "HS256",
        "typ": "JWT"
    }
    
    # JWT Payload
    now = datetime.utcnow()
    exp = now + timedelta(hours=expires_in_hours)
    
    payload = {
        "sub": user_id,  # Subject - the user ID
        "iat": int(now.timestamp()),  # Issued at
        "exp": int(exp.timestamp()),  # Expiration
        "iss": "test-issuer",  # Issuer
        "aud": "smartlock-api"  # Audience
    }
    
    # Encode header and payload
    header_encoded = base64.urlsafe_b64encode(json.dumps(header).encode()).decode().rstrip('=')
    payload_encoded = base64.urlsafe_b64encode(json.dumps(payload).encode()).decode().rstrip('=')
    
    # Create unsigned token (for testing only)
    signature = "test-signature"  # In production, this would be a proper HMAC signature
    
    jwt_token = f"{header_encoded}.{payload_encoded}.{signature}"
    return jwt_token

# Generate JWT token for our test user
JWT_TOKEN = create_test_jwt_token(USER_ID)
print(f"Generated JWT Token: {JWT_TOKEN[:50]}...")
print(f"Token length: {len(JWT_TOKEN)}")

## API Helper Functions

In [ ]:
def make_api_request(endpoint: str, data: Dict[str, Any], 
                    use_auth: bool = True, timeout: int = API_TIMEOUT) -> Dict[str, Any]:
    """
    Make an API request to the specified endpoint.
    
    Args:
        endpoint: API endpoint (without base URL)
        data: Request payload
        use_auth: Whether to include JWT authentication
        timeout: Request timeout in seconds
        
    Returns:
        API response as dictionary
    """
    url = f"{BASE_URL}/{endpoint}"
    headers = {
        "Content-Type": "application/json"
    }
    
    if use_auth:
        headers["X-Original-Authorization"] = f"Bearer {JWT_TOKEN}"
    
    print(f"🚀 Making request to: {url}")
    print(f"📤 Request data: {json.dumps(data, indent=2)}")
    
    try:
        response = requests.post(url, json=data, headers=headers, timeout=timeout)
        
        print(f"📊 Response status: {response.status_code}")
        
        # Try to parse JSON response
        try:
            response_data = response.json()
            print(f"📥 Response data: {json.dumps(response_data, indent=2)}")
        except json.JSONDecodeError:
            response_data = {"raw_response": response.text}
            print(f"📥 Raw response: {response.text}")
        
        return {
            "status_code": response.status_code,
            "success": response.status_code < 400,
            "data": response_data,
            "headers": dict(response.headers)
        }
        
    except requests.exceptions.Timeout:
        print(f"⏰ Request timed out after {timeout} seconds")
        return {
            "status_code": 408,
            "success": False,
            "error": "Request timeout",
            "data": None
        }
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {str(e)}")
        return {
            "status_code": 500,
            "success": False,
            "error": str(e),
            "data": None
        }

def print_section_header(title: str):
    """Print a formatted section header"""
    print("\n" + "="*60)
    print(f"  {title}")
    print("="*60)

## Test Results Storage

In [ ]:
# Dictionary to store test results
test_results = {
    "create_user": None,
    "create_smart_lock": None,
    "add_resident": None,
    "get_user_locks": None,
    "get_lock_residents": None
}

# Variables to store created IDs
created_lock_id = None
created_lock_uuid = None

## 1. Create a User

First, let's create a new user:

In [ ]:
print_section_header("STEP 1: CREATE USER")

create_user_payload = {
    "id": USER_ID,
    "username": USER_USERNAME,
    "email": USER_EMAIL
}

result = make_api_request(
    endpoint="UserRestateService/createUser",
    data=create_user_payload,
    use_auth=False  # User creation might not require auth
)

test_results["create_user"] = result

if result["success"]:
    print("✅ User created successfully!")
else:
    error_msg = str(result.get("data", "")).lower()
    if "already exists" in error_msg or "duplicate" in error_msg:
        print("ℹ️  User already exists, continuing with tests...")
        # Mark as success for test flow purposes
        test_results["create_user"]["success"] = True
    else:
        print("❌ Failed to create user")
        print(f"   Error: {result.get('data', 'Unknown error')}")

## 2. Create a Smart Lock

Now let's create a smart lock for our user:

In [ ]:
print_section_header("STEP 2: CREATE SMART LOCK")

# Generate a UUID for the smart lock
lock_uuid = str(uuid.uuid4())
created_lock_uuid = lock_uuid

create_lock_payload = {
    "id": lock_uuid,
    "name": LOCK_NAME,
    "ownerId": USER_ID,
    "indexNumber": LOCK_INDEX,
    "passcode": LOCK_PASSCODE,
    "serialNumber": f"SL-{generate_random_string(8).upper()}"
}

result = make_api_request(
    endpoint="SmartLockRestateService/createSmartLock",
    data=create_lock_payload,
    use_auth=True
)

test_results["create_smart_lock"] = result

if result["success"]:
    print("✅ Smart lock created successfully!")
    created_lock_id = result["data"]
    print(f"📝 Created lock ID: {created_lock_id}")
else:
    error_msg = str(result.get("data", "")).lower()
    if "already exists" in error_msg or "duplicate" in error_msg:
        print("ℹ️  Smart lock already exists, using existing one...")
        created_lock_id = lock_uuid
        # Mark as success for test flow purposes
        test_results["create_smart_lock"]["success"] = True
    else:
        print("❌ Failed to create smart lock")
        print(f"   Error: {result.get('data', 'Unknown error')}")

## 3. Add a Resident to the Lock

Let's add a resident to our smart lock:

In [ ]:
print_section_header("STEP 3: ADD RESIDENT TO LOCK")

add_resident_payload = {
    "lockIndex": LOCK_INDEX,
    "residentName": RESIDENT_NAME,
    "email": RESIDENT_EMAIL
}

result = make_api_request(
    endpoint="SmartLockRestateService/addResident",
    data=add_resident_payload,
    use_auth=True
)

test_results["add_resident"] = result

if result["success"]:
    print("✅ Resident added successfully!")
else:
    print("❌ Failed to add resident")

## 4. Get All Smart Locks for User

Retrieve all smart locks owned by our user:

In [ ]:
print_section_header("STEP 4: GET USER'S SMART LOCKS")

get_locks_payload = {
    "ownerId": USER_ID  # This will be overridden by auth context
}

result = make_api_request(
    endpoint="SmartLockRestateService/getAllSmartLocks",
    data=get_locks_payload,
    use_auth=True
)

test_results["get_user_locks"] = result

if result["success"]:
    print("✅ Successfully retrieved smart locks!")
    locks = result["data"]
    if isinstance(locks, list):
        print(f"📊 Found {len(locks)} smart lock(s)")
        for i, lock in enumerate(locks):
            print(f"  Lock {i+1}: {lock.get('name', 'N/A')} (ID: {lock.get('id', 'N/A')})")
else:
    print("❌ Failed to retrieve smart locks")

## 5. Get Residents for the Lock

Retrieve all residents for our smart lock:

In [ ]:
print_section_header("STEP 5: GET LOCK RESIDENTS")

get_residents_payload = {
    "lockIndex": LOCK_INDEX
}

result = make_api_request(
    endpoint="SmartLockRestateService/getResidentsByLockIndex",
    data=get_residents_payload,
    use_auth=True
)

test_results["get_lock_residents"] = result

if result["success"]:
    print("✅ Successfully retrieved residents!")
    residents = result["data"]
    if isinstance(residents, list):
        print(f"📊 Found {len(residents)} resident(s)")
        for i, resident in enumerate(residents):
            print(f"  Resident {i+1}: {resident.get('residentName', 'N/A')} ({resident.get('email', 'N/A')})")
else:
    print("❌ Failed to retrieve residents")

## 6. Bonus: Get User Information

Let's also test getting the user information:

In [ ]:
print_section_header("BONUS: GET USER INFORMATION")

get_user_payload = {
    "userId": USER_ID
}

result = make_api_request(
    endpoint="UserRestateService/getUserById",
    data=get_user_payload,
    use_auth=False
)

if result["success"]:
    print("✅ Successfully retrieved user information!")
    user_data = result["data"]
    if user_data:
        print(f"📊 User Details:")
        print(f"  ID: {user_data.get('id', 'N/A')}")
        print(f"  Username: {user_data.get('username', 'N/A')}")
        print(f"  Email: {user_data.get('email', 'N/A')}")
        print(f"  Smart Locks: {len(user_data.get('smartLockIds', []))}")
        print(f"  Keys: {len(user_data.get('keyIds', []))}")
else:
    print("❌ Failed to retrieve user information")

## 🧹 Test Cleanup

Clean up test data to prevent conflicts in future test runs:

In [ ]:
print_section_header("TEST CLEANUP")

cleanup_results = []

def cleanup_resident(lock_id: str, resident_email: str) -> bool:
    """Clean up a resident from a smart lock."""
    try:
        print(f"🧹 Cleaning up resident: {resident_email} from lock: {lock_id}")
        
        cleanup_payload = {
            "smartLockId": lock_id,
            "email": resident_email
        }
        
        result = make_api_request(
            endpoint="SmartLockRestateService/deleteBuildingResident",
            data=cleanup_payload,
            use_auth=True
        )
        
        if result["success"]:
            print(f"✅ Resident {resident_email} deleted successfully")
            return True
        else:
            print(f"⚠️  Failed to delete resident {resident_email}: {result.get('data', 'Unknown error')}")
            return False
            
    except Exception as e:
        print(f"⚠️  Error during resident cleanup: {str(e)}")
        return False

# Cleanup created test data (non-blocking - don't fail tests if cleanup fails)
print("🧹 Starting test data cleanup...")

# Clean up resident if it was created and we have a lock ID
if created_lock_uuid and test_results.get("add_resident", {}).get("success", False):
    cleanup_success = cleanup_resident(created_lock_uuid, RESIDENT_EMAIL)
    cleanup_results.append(("Resident", RESIDENT_EMAIL, cleanup_success))
else:
    print("ℹ️  No resident to clean up (not created or creation failed)")

# Note: Users and SmartLocks don't have delete endpoints in the current API
# Using unique timestamp-based IDs reduces collision probability for future runs

print("\n📋 Cleanup Summary:")
if cleanup_results:
    for item_type, item_id, success in cleanup_results:
        status = "✅ Deleted" if success else "⚠️  Failed"
        print(f"   {item_type}: {item_id} - {status}")
else:
    print("   No cleanup operations performed")

print("\n💡 Future Collision Prevention:")
print(f"   Using timestamp-based IDs: {timestamp}")
print(f"   Lock Index: {LOCK_INDEX} (timestamp-derived)")
print(f"   User ID: {USER_ID} (unique per run)")
print("✅ Cleanup completed")

## Test Summary and Results Analysis

In [ ]:
print_section_header("TEST SUMMARY")

# Create a summary table
summary_data = []
for test_name, result in test_results.items():
    if result:
        status = "✅ PASS" if result["success"] else "❌ FAIL"
        status_code = result["status_code"]
        error = result.get("error", "None")
    else:
        status = "⏭️ SKIP"
        status_code = "N/A"
        error = "Not executed"
    
    summary_data.append({
        "Test": test_name.replace("_", " ").title(),
        "Status": status,
        "HTTP Code": status_code,
        "Error": error
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Calculate success rate
total_tests = len([r for r in test_results.values() if r is not None])
successful_tests = len([r for r in test_results.values() if r and r["success"]])
success_rate = (successful_tests / total_tests * 100) if total_tests > 0 else 0

print(f"\n📊 Success Rate: {successful_tests}/{total_tests} ({success_rate:.1f}%)")

# Show test configuration used
print(f"\n🔧 Test Configuration Used:")
config_data = [
    ["User ID", USER_ID],
    ["User Email", USER_EMAIL],
    ["User Username", USER_USERNAME],
    ["Lock Name", LOCK_NAME],
    ["Lock Index", LOCK_INDEX],
    ["Lock UUID", created_lock_uuid or "Not created"],
    ["Resident Name", RESIDENT_NAME],
    ["Resident Email", RESIDENT_EMAIL],
    ["Base URL", BASE_URL]
]

config_df = pd.DataFrame(config_data, columns=["Parameter", "Value"])
print(config_df.to_string(index=False))

## Additional Test Utilities

Helper functions for running individual tests or custom scenarios:

In [ ]:
def test_get_smart_lock_by_id(lock_id: str):
    """Test getting a specific smart lock by ID"""
    print(f"🔍 Testing get smart lock by ID: {lock_id}")
    
    payload = {"smartLockId": lock_id}
    result = make_api_request(
        endpoint="SmartLockRestateService/getSmartLockById",
        data=payload,
        use_auth=True
    )
    return result

def test_get_smart_lock_by_index(index: int):
    """Test getting a specific smart lock by index"""
    print(f"🔍 Testing get smart lock by index: {index}")
    
    payload = {"indexNumber": index}
    result = make_api_request(
        endpoint="SmartLockRestateService/getSmartLockByIndex",
        data=payload,
        use_auth=True
    )
    return result

def test_get_user_by_email(email: str):
    """Test getting a user by email"""
    print(f"🔍 Testing get user by email: {email}")
    
    payload = {"email": email}
    result = make_api_request(
        endpoint="UserRestateService/getUserByEmail",
        data=payload,
        use_auth=False
    )
    return result

def test_custom_scenario():
    """Run custom test scenarios here"""
    print("🧪 Running custom test scenarios...")
    
    # Example: Test getting smart lock by ID if we have one
    if created_lock_uuid:
        result = test_get_smart_lock_by_id(created_lock_uuid)
        print(f"Get lock by ID result: {result['success']}")
    
    # Example: Test getting smart lock by index
    result = test_get_smart_lock_by_index(LOCK_INDEX)
    print(f"Get lock by index result: {result['success']}")
    
    # Example: Test getting user by email
    result = test_get_user_by_email(USER_EMAIL)
    print(f"Get user by email result: {result['success']}")

# Uncomment the line below to run custom scenarios
# test_custom_scenario()

## Performance Analysis

Analyze response times and performance metrics:

In [ ]:
def benchmark_endpoint(endpoint: str, payload: dict, iterations: int = 3):
    """Benchmark an endpoint with multiple requests"""
    print(f"⏱️  Benchmarking {endpoint} with {iterations} iterations...")
    
    response_times = []
    success_count = 0
    
    for i in range(iterations):
        start_time = time.time()
        result = make_api_request(endpoint, payload, use_auth=True)
        end_time = time.time()
        
        response_time = end_time - start_time
        response_times.append(response_time)
        
        if result["success"]:
            success_count += 1
        
        print(f"  Iteration {i+1}: {response_time:.2f}s - {'✅' if result['success'] else '❌'}")
        time.sleep(1)  # Small delay between requests
    
    avg_time = sum(response_times) / len(response_times)
    success_rate = (success_count / iterations) * 100
    
    print(f"📊 Average response time: {avg_time:.2f}s")
    print(f"📊 Success rate: {success_rate:.1f}%")
    print(f"📊 Min/Max time: {min(response_times):.2f}s / {max(response_times):.2f}s")
    
    return {
        "avg_time": avg_time,
        "success_rate": success_rate,
        "response_times": response_times
    }

# Example: Benchmark getting all smart locks
# benchmark_result = benchmark_endpoint(
#     "SmartLockRestateService/getAllSmartLocks",
#     {"ownerId": USER_ID},
#     iterations=3
# )

## Cleanup (Optional)

If the API supports deletion operations, you can add cleanup code here:

In [ ]:
def cleanup_test_data():
    """Cleanup test data - implement if deletion endpoints are available"""
    print("🧹 Cleanup functionality not implemented yet")
    print("   (API deletion endpoints would be needed)")
    print(f"   Created test data:")
    print(f"   - User ID: {USER_ID}")
    print(f"   - Lock UUID: {created_lock_uuid}")
    print(f"   - Lock Index: {LOCK_INDEX}")

# Uncomment to run cleanup
# cleanup_test_data()

## Conclusion

This notebook demonstrates the complete API testing flow:

1. ✅ **User Creation**: Creates a new user with random data
2. ✅ **JWT Authentication**: Generates test JWT tokens for API authentication
3. ✅ **Smart Lock Management**: Creates smart locks and associates them with users
4. ✅ **Resident Management**: Adds residents to smart locks
5. ✅ **Data Retrieval**: Tests various GET endpoints
6. ✅ **Error Handling**: Properly handles timeouts and errors
7. ✅ **Performance Monitoring**: Provides timing and success rate analysis

### Customization

To customize this notebook for different test scenarios:

1. **Modify the configuration section** to use specific test data
2. **Adjust the JWT token generation** if your authentication requirements differ
3. **Add new test functions** for additional endpoints
4. **Use the benchmark functions** to test performance under load

### Notes

- The JWT tokens generated here are for testing purposes and use a simple format
- All test data uses random values by default to avoid conflicts
- The notebook handles both successful responses and various error conditions
- Response times and success rates are tracked for performance analysis

In [ ]:
# Colab-specific results summary
print("\n" + "="*80)
print("🎉 GOOGLE COLAB TEST SUMMARY")
print("="*80)

# Calculate final statistics
total_tests = len([t for t in test_results.values() if t is not None])
passed_tests = len([t for t in test_results.values() if t and t.get("success", False)])
failed_tests = total_tests - passed_tests
success_rate = (passed_tests/total_tests*100) if total_tests > 0 else 0

print(f"📊 Test Results:")
print(f"   ✅ Passed: {passed_tests}/{total_tests} ({success_rate:.1f}%)")
print(f"   🌐 Environment: {ENVIRONMENT} ({BASE_URL})")
print(f"   ⏱️ Session Duration: {(datetime.now() - test_metadata['start_time']).total_seconds():.1f}s")

if success_rate >= 100:
    print(f"\n🎉 EXCELLENT! All tests passed. The SmartLock API is working perfectly!")
elif success_rate >= 80:
    print(f"\n✅ GOOD! Most tests passed. Minor issues detected.")
else:
    print(f"\n⚠️ ATTENTION! Several tests failed. Please check the details above.")

print(f"\n💡 Tips:")
print(f"   • Re-run this notebook anytime to test again")
print(f"   • Each run uses unique test data to avoid conflicts")
print(f"   • Try different environments using the configuration above")
print(f"   • Download results using the export cell if needed")

print(f"\n🔗 Useful Links:")
print(f"   📄 This notebook: https://colab.research.google.com/github/{os.getenv('COLAB_GITHUB_REPO', 'your-org/repo')}/blob/main/smartlock_api_testing_colab.ipynb")
print(f"   📚 API Documentation: [Contact support for API docs]")
print(f"   🐛 Report Issues: [Contact support for issue reporting]")
print(f"   💬 Support: [Contact support team]")

print(f"\n🤖 This notebook was automatically generated and updated by CI/CD")
print(f"   Last update: {datetime.now().strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"   Source commit: {os.getenv('CI_COMMIT_SHA', 'unknown')}")
print("="*80)


In [ ]:
#@title 💾 Export Test Results (Optional) { display-mode: "form" }
#@markdown Download your test results as a JSON file:

EXPORT_RESULTS = False #@param {type:"boolean"}

if EXPORT_RESULTS:
    # Prepare export data
    export_data = {
        "metadata": {
            "test_session": "Google Colab",
            "start_time": test_metadata["start_time"].isoformat(),
            "end_time": datetime.now().isoformat(),
            "environment": ENVIRONMENT,
            "base_url": BASE_URL,
            "user_id": USER_ID
        },
        "summary": {
            "total_tests": len([t for t in test_results.values() if t is not None]),
            "passed_tests": len([t for t in test_results.values() if t and t.get("success", False)]),
            "success_rate": (len([t for t in test_results.values() if t and t.get("success", False)]) / len([t for t in test_results.values() if t is not None]) * 100) if len([t for t in test_results.values() if t is not None]) > 0 else 0
        },
        "test_results": test_results,
        "test_data": {
            "user_email": USER_EMAIL,
            "lock_name": LOCK_NAME,
            "lock_index": LOCK_INDEX
        }
    }
    
    # Convert to JSON
    import json
    export_json = json.dumps(export_data, indent=2, default=str)
    
    # Save and download in Colab
    filename = f"smartlock_api_test_results_{ENVIRONMENT}_{int(time.time())}.json"
    with open(filename, 'w') as f:
        f.write(export_json)
    
    print(f"💾 Results exported to: {filename}")
    
    # Download file in Colab
    try:
        from google.colab import files
        files.download(filename)
        print(f"📥 File downloaded to your computer!")
    except ImportError:
        print(f"📁 File saved locally: {filename}")
        
else:
    print("💡 Enable 'Export Results' above to download test results as JSON")
